# AutoGluon: Porto Seguro (Classification) # 2

Based on: https://auto.gluon.ai/0.1.0/tutorials/tabular_prediction/tabular-indepth.html

In [1]:
time_limit = 60*60  # 1 hour max (confirm with timer)
metric = "roc_auc"  # specify your evaluation metric
label = "target" # name of target or label variable (just happens to be called "target" for this dataset)

## Import

In [2]:
# Import packages

from autogluon.tabular import TabularDataset
from autogluon.tabular import TabularPredictor

import numpy as np
import pandas as pd

In [3]:
# Import data

train_data = TabularDataset("porto_train.csv")
test_data = TabularDataset("porto_test.csv")

In [4]:
train_data.shape

(476170, 60)

In [5]:
test_data.shape

(119042, 59)

## Tidy

In [6]:
train_data = train_data.replace(-1, np.nan)
test_data = test_data.replace(-1, np.nan)

## Transform

In [7]:
# Drop variable not used as predictors
train_data = train_data.drop(["id", "fold"], axis=1)
test_data = test_data.drop(["id"], axis=1)

In [8]:
# Convert categorical variables to "category" data type
cat_vars = [col for col in train_data.columns if 'cat' in col]
cat_vars

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [9]:
for col in cat_vars:
    test_data[col] = test_data[col].astype('category')
    
cat_vars = cat_vars + ["target"]

for col in cat_vars:
    train_data[col] = train_data[col].astype('category')

In [10]:
# Create separate test data to demonstrate how to make predictions on new examples at inference time:
y_test = test_data[label]  # values to predict

## Visualize

(skipped)

## Model 1

In [11]:
%timeit

predictor = TabularPredictor(label=label, eval_metric=metric).fit(train_data, time_limit=time_limit)

No path specified. Models will be saved in: "AutogluonModels/ag-20230709_222439/"
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230709_222439/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    140389.35 MB
	T

In [12]:
# Use trained models to make predictions on the new data and then evaluate performance:
y_pred = predictor.predict_proba(test_data)

In [13]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Evaluation: roc_auc on test data: 0.6318303042376591
Evaluations on test data:
{
    "roc_auc": 0.6318303042376591,
    "accuracy": 0.9635842811780716,
    "balanced_accuracy": 0.5,
    "mcc": 0.0,
    "f1": 0.0,
    "precision": 0.0,
    "recall": 0.0
}


In [14]:
# Evaluate the performance of each individual trained model on test data:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.635058,0.602004,0.252682,0.025506,52.114292,0.252682,0.025506,52.114292,1,True,7
1,XGBoost,0.634824,0.601470,0.923697,0.037446,7.272154,0.923697,0.037446,7.272154,1,True,11
2,WeightedEnsemble_L2,0.631830,0.612350,4.878562,0.350731,684.024112,0.009778,0.001179,1.383185,2,True,14
3,NeuralNetFastAI,0.623164,0.590777,1.425133,0.066629,345.052452,1.425133,0.066629,345.052452,1,True,10
4,LightGBMXT,0.621983,0.591881,0.137941,0.019865,4.372670,0.137941,0.019865,4.372670,1,True,3
5,LightGBMLarge,0.621455,0.587142,0.130643,0.012186,3.290614,0.130643,0.012186,3.290614,1,True,13
6,NeuralNetTorch,0.620740,0.594228,1.110789,0.062991,266.680689,1.110789,0.062991,266.680689,1,True,12
7,LightGBM,0.617411,0.581836,0.117229,0.013983,2.019260,0.117229,0.013983,2.019260,1,True,4
8,ExtraTreesEntr,0.612614,0.601840,1.156483,0.156979,11.521340,1.156483,0.156979,11.521340,1,True,9
9,ExtraTreesGini,0.609756,0.585093,1.833930,0.156187,11.101402,1.833930,0.156187,11.101402,1,True,8


## Model 2

In [15]:
%timeit

# num_bag_folds=5, num_bag_sets=1, num_stack_levels=3,  # takes too long?
    
predictor = TabularPredictor(
    label=label, 
    eval_metric=metric
).fit(
    train_data,
    time_limit=time_limit,
    num_bag_folds=5, 
    num_bag_sets=1, 
    num_stack_levels=1,
    hyperparameters = {'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}} 
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230709_223755/"
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230709_223755/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    131768.69 MB
	T

TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20230709_223755/")


In [16]:
# Use trained models to make predictions on the new data and then evaluate performance:
y_pred = predictor.predict_proba(test_data)

2023-07-09 22:38:44,421	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-07-09 22:38:44,422	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-07-09 22:38:44,426	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2023-07-09 22:38:44,435	ERROR worker.py:400 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.


In [17]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Evaluation: roc_auc on test data: 0.6182609623441678
Evaluations on test data:
{
    "roc_auc": 0.6182609623441678,
    "accuracy": 0.9635842811780716,
    "balanced_accuracy": 0.5,
    "mcc": 0.0,
    "f1": 0.0,
    "precision": 0.0,
    "recall": 0.0
}


In [18]:
# Evaluate the performance of each individual trained model on test data:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.618261,0.622848,0.481776,0.641664,4.802759,0.481776,0.641664,4.802759,1,True,1
1,WeightedEnsemble_L2,0.618261,0.622848,0.486474,0.735487,4.906338,0.004699,0.093822,0.103579,2,True,2
2,LightGBM_BAG_L2,0.617080,0.618365,1.019338,1.214663,9.333133,0.537562,0.572998,4.530375,2,True,3
3,WeightedEnsemble_L3,0.617080,0.618365,1.024849,1.294668,9.416901,0.005511,0.080006,0.083767,3,True,4


## Model 3

In [19]:
%timeit

predictor = TabularPredictor(
    label=label, 
    eval_metric=metric
).fit(
    train_data,
    time_limit=time_limit,
    auto_stack=True,
    hyperparameters={'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}} 
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230709_223846/"
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230709_223846/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fit

Fitting model: LightGBM_BAG_L1 ... Training model for up to 3519.1s of the 3519.1s of remaining time.
	Fitting 8 child models (S7F1 - S7F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6257	 = Validation score   (roc_auc)
	36.54s	 = Training   runtime
	5.95s	 = Validation runtime
Repeating k-fold bagging: 8/20
Fitting model: LightGBM_BAG_L1 ... Training model for up to 3506.42s of the 3506.42s of remaining time.
	Fitting 8 child models (S8F1 - S8F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6256	 = Validation score   (roc_auc)
	42.02s	 = Training   runtime
	6.79s	 = Validation runtime
Repeating k-fold bagging: 9/20
Fitting model: LightGBM_BAG_L1 ... Training model for up to 3493.54s of the 3493.54s of remaining time.
	Fitting 8 child models (S9F1 - S9F8) | Fitting with ParallelLocalFoldFittingStrategy
	0.6257	 = Validation score   (roc_auc)
	47.65s	 = Training   runtime
	7.68s	 = Validation runtime
Repeating k-fold bagging: 10/20
Fitting model: LightGBM_BAG_L1 ... Traini

In [20]:
# Use trained models to make predictions on the new data and then evaluate performance:
y_pred = predictor.predict_proba(test_data)

In [21]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Evaluation: roc_auc on test data: 0.618060270483639
Evaluations on test data:
{
    "roc_auc": 0.618060270483639,
    "accuracy": 0.9635842811780716,
    "balanced_accuracy": 0.5,
    "mcc": 0.0,
    "f1": 0.0,
    "precision": 0.0,
    "recall": 0.0
}


In [22]:
# Evaluate the performance of each individual trained model on test data:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.61806,0.625769,16.278193,16.827170,104.648937,16.278193,16.827170,104.648937,1,True,1
1,WeightedEnsemble_L2,0.61806,0.625769,16.283676,16.921221,104.743850,0.005484,0.094052,0.094913,2,True,2


Note: specifying `presets='best_quality'` in `fit()` simply sets `auto_stack=True`

## Model 4

In [23]:
%timeit

presets = ['good_quality_faster_inference_only_refit', 'optimize_for_deployment']

predictor = TabularPredictor(
    label=label, 
    eval_metric=metric
).fit(
    train_data, 
    presets=presets, 
    time_limit=time_limit
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230709_224341/"
Preset alias specified: 'good_quality_faster_inference_only_refit' maps to 'good_quality'.
Presets specified: ['good_quality_faster_inference_only_refit', 'optimize_for_deployment']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230709_224341/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specif

Deleting model LightGBMXT_BAG_L1. All files under AutogluonModels/ag-20230709_224341/models/LightGBMXT_BAG_L1/ will be removed.
Deleting model LightGBM_BAG_L1. All files under AutogluonModels/ag-20230709_224341/models/LightGBM_BAG_L1/ will be removed.
Deleting model RandomForestGini_BAG_L1. All files under AutogluonModels/ag-20230709_224341/models/RandomForestGini_BAG_L1/ will be removed.
Deleting model RandomForestEntr_BAG_L1. All files under AutogluonModels/ag-20230709_224341/models/RandomForestEntr_BAG_L1/ will be removed.
Deleting model CatBoost_BAG_L1. All files under AutogluonModels/ag-20230709_224341/models/CatBoost_BAG_L1/ will be removed.
Deleting model ExtraTreesGini_BAG_L1. All files under AutogluonModels/ag-20230709_224341/models/ExtraTreesGini_BAG_L1/ will be removed.
Deleting model ExtraTreesEntr_BAG_L1. All files under AutogluonModels/ag-20230709_224341/models/ExtraTreesEntr_BAG_L1/ will be removed.
Deleting model NeuralNetFastAI_BAG_L1. All files under AutogluonModels/a

In [24]:
# Use trained models to make predictions on the new data and then evaluate performance:
y_pred = predictor.predict_proba(test_data)

In [25]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Evaluation: roc_auc on test data: 0.6365084165243277
Evaluations on test data:
{
    "roc_auc": 0.6365084165243277,
    "accuracy": 0.9635842811780716,
    "balanced_accuracy": 0.5,
    "mcc": 0.0,
    "f1": 0.0,
    "precision": 0.0,
    "recall": 0.0
}


In [26]:
# Evaluate the performance of each individual trained model on test data:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2_FULL,0.636508,None,5.075468,NaN,350.777234,0.014975,NaN,92.928672,2,True,10
1,CatBoost_BAG_L1_FULL,0.636259,None,0.197843,NaN,72.327449,0.197843,NaN,72.327449,1,True,5
2,XGBoost_BAG_L1_FULL,0.632122,None,0.761734,NaN,7.261549,0.761734,NaN,7.261549,1,True,7
3,NeuralNetFastAI_BAG_L1_FULL,0.629000,None,1.342263,NaN,92.096926,1.342263,NaN,92.096926,1,True,6
4,RandomForestEntr_BAG_L1_FULL,0.626495,None,0.560144,23.397590,14.958012,0.560144,23.397590,14.958012,1,True,4
5,LightGBMXT_BAG_L1_FULL,0.625159,None,0.144905,NaN,2.388774,0.144905,NaN,2.388774,1,True,1
6,LightGBMLarge_BAG_L1_FULL,0.624381,None,0.161299,NaN,3.648124,0.161299,NaN,3.648124,1,True,9
7,LightGBM_BAG_L1_FULL,0.623333,None,0.140113,NaN,2.151647,0.140113,NaN,2.151647,1,True,2
8,RandomForestGini_BAG_L1_FULL,0.623265,None,0.531812,25.453383,15.243395,0.531812,25.453383,15.243395,1,True,3
9,NeuralNetTorch_BAG_L1_FULL,0.620621,None,1.220380,NaN,47.772686,1.220380,NaN,47.772686,1,True,8


## Model 5

In [27]:
%timeit

predictor_light = TabularPredictor(
    label=label, 
    eval_metric=metric
).fit(
    train_data,
    time_limit=time_limit,
    hyperparameters='very_light'
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230709_232646/"
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230709_232646/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    123462.25 MB
	T

In [28]:
# Use trained models to make predictions on the new data and then evaluate performance:
y_pred = predictor.predict_proba(test_data)

In [29]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Evaluation: roc_auc on test data: 0.6365084165243277
Evaluations on test data:
{
    "roc_auc": 0.6365084165243277,
    "accuracy": 0.9635842811780716,
    "balanced_accuracy": 0.5,
    "mcc": 0.0,
    "f1": 0.0,
    "precision": 0.0,
    "recall": 0.0
}


In [30]:
# Evaluate the performance of each individual trained model on test data:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2_FULL,0.636508,None,5.299780,NaN,350.777234,0.013492,NaN,92.928672,2,True,10
1,CatBoost_BAG_L1_FULL,0.636259,None,0.180383,NaN,72.327449,0.180383,NaN,72.327449,1,True,5
2,XGBoost_BAG_L1_FULL,0.632122,None,1.016783,NaN,7.261549,1.016783,NaN,7.261549,1,True,7
3,NeuralNetFastAI_BAG_L1_FULL,0.629000,None,1.398500,NaN,92.096926,1.398500,NaN,92.096926,1,True,6
4,RandomForestEntr_BAG_L1_FULL,0.626495,None,0.525140,23.397590,14.958012,0.525140,23.397590,14.958012,1,True,4
5,LightGBMXT_BAG_L1_FULL,0.625159,None,0.160896,NaN,2.388774,0.160896,NaN,2.388774,1,True,1
6,LightGBMLarge_BAG_L1_FULL,0.624381,None,0.149963,NaN,3.648124,0.149963,NaN,3.648124,1,True,9
7,LightGBM_BAG_L1_FULL,0.623333,None,0.147710,NaN,2.151647,0.147710,NaN,2.151647,1,True,2
8,RandomForestGini_BAG_L1_FULL,0.623265,None,0.523249,25.453383,15.243395,0.523249,25.453383,15.243395,1,True,3
9,NeuralNetTorch_BAG_L1_FULL,0.620621,None,1.183665,NaN,47.772686,1.183665,NaN,47.772686,1,True,8


## Model 6

In [31]:
%timeit
    
excluded_model_types = ['KNN', 'NN', 'custom']

predictor = TabularPredictor(
    label=label, 
    eval_metric=metric
).fit(
    train_data,
    time_limit=time_limit,
    excluded_model_types=excluded_model_types
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230709_233844/"
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230709_233844/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu May 4 15:21:22 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    123358.18 MB
	T

In [32]:
# Use trained models to make predictions on the new data and then evaluate performance:
y_pred = predictor.predict_proba(test_data)

In [33]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Evaluation: roc_auc on test data: 0.6318303042376591
Evaluations on test data:
{
    "roc_auc": 0.6318303042376591,
    "accuracy": 0.9635842811780716,
    "balanced_accuracy": 0.5,
    "mcc": 0.0,
    "f1": 0.0,
    "precision": 0.0,
    "recall": 0.0
}


In [34]:
# Evaluate the performance of each individual trained model on test data:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.635058,0.602004,0.234534,0.025685,51.196715,0.234534,0.025685,51.196715,1,True,5
1,XGBoost,0.634824,0.601470,0.928967,0.036806,7.742992,0.928967,0.036806,7.742992,1,True,9
2,WeightedEnsemble_L2,0.631830,0.612350,5.244228,0.360610,723.056895,0.010540,0.001154,1.176512,2,True,12
3,NeuralNetFastAI,0.623164,0.590777,1.651537,0.072877,370.594512,1.651537,0.072877,370.594512,1,True,8
4,LightGBMXT,0.621983,0.591881,0.120918,0.025780,2.080570,0.120918,0.025780,2.080570,1,True,1
5,LightGBMLarge,0.621455,0.587142,0.129459,0.013053,3.178371,0.129459,0.013053,3.178371,1,True,11
6,NeuralNetTorch,0.620740,0.594228,1.209033,0.065241,281.466654,1.209033,0.065241,281.466654,1,True,10
7,LightGBM,0.617411,0.581836,0.107351,0.012496,1.850031,0.107351,0.012496,1.850031,1,True,2
8,ExtraTreesEntr,0.612614,0.601840,1.209616,0.158848,10.879509,1.209616,0.158848,10.879509,1,True,7
9,ExtraTreesGini,0.609756,0.585093,1.990415,0.150558,11.031484,1.990415,0.150558,11.031484,1,True,6


## Communicate

(skipped)